# Use `xarray` to extract a time series at a point
## By: Lejo Flores
## April 29, 2019

This Jupyter notebook demonstrates how to use `xarray` to extract a time series of data at a single point specified by a given lat/long coordinate. In particular, we use the `xarray.Dataset.interp()` method to linearly interpolate our dataset to a given point. This example was based on the xarray documentation [found here](http://xarray.pydata.org/en/stable/interpolation.html#example) and by following the example provided in this [StackOverflow post](https://stackoverflow.com/questions/40256461/reading-time-series-from-netcdf-with-python).

### 1. Import the libraries needed and define variables

In [1]:
import xarray as xr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import osgeo
import pandas

from osgeo import ogr
from osgeo import osr
from pandas.plotting import register_matplotlib_converters # Needed to avoid matplotlib warning

# Print versions of libraries used in this notebook for traceability
print("\nPackage version numbers:\n")
print("xarray version: " + xr.__version__)
print("numpy version: " + np.__version__)
print("matplotlib version: " + matplotlib.__version__)
print("osgeo version: " + osgeo.__version__)
print("pandas version: " + pandas.__version__+"\n")

Daymet_path = '/Users/lejoflores/data/daymet/' # Path to Daymet data on my drive
Daymet_name = 'tmax_1980-2018.nc' # Name of the maximum daily temperature data 
Daymet_file = Daymet_path + Daymet_name # Full name of file with path

# Coordinates of Wyoming Big Sage site in Reynolds Creek CZO
lati = 43.167545
loni = -116.713205


ImportError: dlopen(/Users/lejoflores/miniconda3/lib/python3.7/site-packages/osgeo/_gdal.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libpoppler.76.dylib
  Referenced from: /Users/lejoflores/miniconda3/lib/libgdal.20.dylib
  Reason: image not found

### 2. Open the NetCDF dataset using xarray and print its characteristics

In [ ]:
ds = xr.open_dataset(Daymet_file)
print(ds)

### 3. Now print characteristics of the dataset spatial projection

In [ ]:
print(ds['lambert_conformal_conic'])

### 4. Reproject the input point to the desired projection using GDAL

In [ ]:
InSR = osr.SpatialReference()
InSR.ImportFromEPSG(4326) # WGS84/Geographic
OutSR = osr.SpatialReference()
OutSR.ImportFromEPSG(102009) # Lambert Conformal Conic North America 

# Note: these parameters have to be set because they are different from the default values
OutSR.SetProjParm("Central_Meridian", -100.0)
OutSR.SetProjParm("Latitude_Of_Origin", 42.5)
OutSR.SetProjParm("Standard_Parallel_1", 25.0)
OutSR.SetProjParm("Standard_Parallel_2", 60.0)

Point = ogr.Geometry(ogr.wkbPoint)
Point.AddPoint(loni,lati) 
Point.AssignSpatialReference(InSR) # Specify the spatial reference of the input Lat/long coords
Point.TransformTo(OutSR) # Now transform it to projected coordinates 

# Get the projected x and y coordinates 
yi = Point.GetY()
xi = Point.GetX()

### 5. Use the `xarray.Dataset.interp()` to interpolate to the projected point
Note, this creates a new `xarray` dataset that is interpolated to the x, y point input using a linear interpolation. Print the characteristics of this new dataset

In [ ]:
dsloc = ds.interp(x=xi, y=yi)
print(dsloc)

### 6. Get the interpolated time series of temperature at the point and plot

In [ ]:
Tmax = dsloc['tmax'].values
time = dsloc['time'].values

register_matplotlib_converters() # Needed to avoid a matplotlib warning

plt.figure(figsize=(10,6))
plt.rcParams.update({'font.size': 12})
plt.plot(time, Tmax)
plt.title('Daily Maximum Air Temperature at Wyoming Big Sage Site from Daymet')
plt.xlabel('Time')
plt.ylabel('$T_{max} {}^{\circ}C$')
plt.show()